# Special Permissions

Suppose we wish to change our password.  Let's take a look at the system password file and see if we have permission to do so.

In [3]:
ls -l /etc/passwd

-rw-r--r--. 1 root root 2801 Jun  3  2019 /etc/passwd


We are not 'root' and we are not in the 'root' group, so the last three permissions apply to us:
    r--
That means we only have read permission.  So we can look at the file, but can't edit the file to change our password.  Let's see what's in the file:

In [4]:
cat /etc/passwd

root:x:0:0:root:/root:/bin/bash
bin:x:1:1:bin:/bin:/sbin/nologin
daemon:x:2:2:daemon:/sbin:/sbin/nologin
adm:x:3:4:adm:/var/adm:/sbin/nologin
lp:x:4:7:lp:/var/spool/lpd:/sbin/nologin
sync:x:5:0:sync:/sbin:/bin/sync
shutdown:x:6:0:shutdown:/sbin:/sbin/shutdown
halt:x:7:0:halt:/sbin:/sbin/halt
mail:x:8:12:mail:/var/spool/mail:/sbin/nologin
operator:x:11:0:operator:/root:/sbin/nologin
games:x:12:100:games:/usr/games:/sbin/nologin
ftp:x:14:50:FTP User:/var/ftp:/sbin/nologin
nobody:x:99:99:Nobody:/:/sbin/nologin
pegasus:x:66:65:tog-pegasus OpenPegasus WBEM/CIM services:/var/lib/Pegasus:/sbin/nologin
systemd-network:x:192:192:systemd Network Management:/:/sbin/nologin
dbus:x:81:81:System message bus:/:/sbin/nologin
polkitd:x:999:998:User for polkitd:/:/sbin/nologin
unbound:x:998:995:Unbound DNS resolver:/etc/unbound:/sbin/nologin
saslauth:x:997:76:Saslauthd user:/run/saslauthd:/sbin/nologin
libstoragemgmt:x:996:994:daemon account for libstoragemgmt:/var/run/lsm:/sbin/nologin
rpc:x:32:32:Rpcb

The password used to be in field two, but now there is only an 'x'.  That's because the password has been moved to the shadow file:

In [6]:
ls -l /etc/shadow

----------. 1 root root 1333 Jun  3  2019 /etc/shadow


We don't even have read permission on this file (for security reasons).  So how can we change our password?
Linux employs a special mechanism to allow us to perform the change.  We can't edit the 'shadow' file directly, but we can use a program called 'passwd' to do it for us:

In [7]:
ls -l /usr/bin/passwd

-rwsr-xr-x. 1 root root 27936 Mar 22  2019 /usr/bin/passwd*


Note the first three permissions: rws.  The 's' stands for special permission.  A program with 's' permission is called a setuid program.  What it means is that anyone running this program will temporarily become root (the owner of the setuid program).  We know root has write access to the '/etc/passwd' file and hence we will be allowed to change our password with this program.  

Linux has a concept of 'real' and 'effective' user ids.  By running a setuid program our 'real' userid doesn't change, but our 'effective' userid does.  Linux permissions are based on effective userid and not on real userid.  We say we impersonate 'root' while running this program.  

The passwd program adds an extra level of security and will only allow us to change our own password; the program knows who we are because of our real userid.

Linux also defines a setgroupid program where an 's' is specified in the group permissions.  This works in the same way as the setuid, but applies to anyone in the group. The 's' permission allows the 'real' user to impersonate the owner of the program - in this case 'root'.

There is one othe special permission.  This is the sticky bit 't'.  This has nothing to do with the 's' permissions.  

Consider the case where we are editing a file with 'vim'.  For large files 'vim' stores its working data in the directory: /var/temp.  

However, since anyone can use 'vim', everyone needs write access to this directory:

In [9]:
ls -ld /var/tmp

drwxrwxrwt. 15 root root 4096 Feb 11 20:18 /var/tmp/


We can see we do have write access to this directory, so all is well.  However there is a problem with this scheme.  It was found that by giving everyone access to this folder it meant that malicious users could delete othe user's temporary data.  This used to happen a lot in a University environment, when students regarded this a a practical joke.  Clearly, this defect needs to be addressed.  

Notice the 't' bit in the permissions for this directory.  This makes the directory 'sticky'.  In a sticky directory users are only allowed to delete their own files even though they write permission.  The 'sticky' bit effectively controls how the write permission works.

You might be wondering why the 't' bit is called the 'sticky' bit.  Well, in the early days of Unix the sticky bit was used to keep a program in memory (stick in memory).  If a program was stuck in memory it would run faster than other programs (they would be swapped out to disk when their time slice ended).  As Unix became more sophisticated this mechanism was replaced by paging in and out virtual memory and the 'sticky' bit was no longer required.  Since the 't' permission was available it was reassigned as above, but the name wasn't changed.